# Прогноз необходимого количества трафика для AB теста
Учитывается неравное деление трафика по группам

In [1]:
import scipy.stats as scs
import pandas as pd
import numpy as np

In [2]:
# Задаем параметры

# bcr - словарь текущих показателей отслеживаемых показателей

bcrs = {
    'CR из пользователя в просмотр карточки товара': 0.3,
    'CR из пользователя в добавление в корзину': 0.1,
    'CR из пользователя в переход в корзину': 0.05,
    'CR из пользователя в покупателя': 0.02
}

# oжидаемый uplift отслеживаемого показателя, относительная величина - на сколько процентов хотим увеличить наш показатель
# можно сформировать список из нескольких вариантов, по которым хотим проверить
uplifts = [0.01, 0.02, 0.03, 0.05, 0.07, 0.10, 0.15, 0.2]

# количество пользователей в день
users_day = 1000

# мощность теста
power=0.8

# минимальный уровень стат.значимости
pvalue=0.05

# соотношение трафика, если 50%/50%, то 0.5, 90%/10%, то 0.9, и т.д.
ratio=0.5

In [3]:
def min_sample_size(bcrs, uplifts, users_day, power, pvalue, ratio):
    """
    Функция возвращает минимальный размер выборки для настройки AB-теста
     Аргументы:
         bcr (float): базовая конверсия
         mde (float): минимальный uplift, который ожидаем получить
         users_day () - количество пользователей в день
         power (float): вероятность отклонения нулевой гипотезы, когда
         нулевая гипотеза неверна, обычно 0,8
         sig_level (float): уровень значимости часто обозначается как альфа,
         обычно 0,05
     Возвращает:
         датафрейм с прогнозом продолжительности AB теста по каждому показателю по каждому uplift
     Ссылки:
         Стэнфордская лекция о размерах выборки: http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
         Калькулятор Эвана Миллера для расчета продолжительности теста: https://www.evanmiller.org/ab-testing/sample-size.html
    """
    
    #################################################################
    # набор колонок для датафрейма имходя из нужных uplift
    columns = ['conversion_name','current_rate', 'users_day', 'power', 'pvalue', 'ratio']
    for item in uplifts:
        column_temp = 'uplift '+str(round(item*100, 2))+'%'
        columns.append(column_temp)
    result = pd.DataFrame(columns = columns)
    #################################################################
    
    marks = bcrs.keys()
    for item in marks:
        
        # массив значений для строки таблицы
        result_calculate = []
        
        # заполняем массив базовыми значениями
        cr_name = item
        bcr = bcrs[item]
        result_calculate.append(item)
        result_calculate.append(bcr)
        result_calculate.append(users_day)
        result_calculate.append(power)
        result_calculate.append(pvalue)
        result_calculate.append(ratio)
        
        # считаем необходимое количество дней для каждого uplift
        for uplift in uplifts:
            mde = bcr+bcr*uplift
            standard_norm = scs.norm(0, 1)
            Z_beta = standard_norm.ppf(power)
            Z_alpha = standard_norm.ppf(1-pvalue/2) # если у нас двусторонний тест, делим на 1, если двусторонний, делим на 2


            min_N = (((Z_beta + Z_alpha)**2)*bcr*(1-bcr)) / ((mde-bcr)**2 * ratio*(1-ratio))
            
            # необходимое количество дней
            calc_days = int(round(min_N/users_day))
            if calc_days == 0: count_days = 1
            else: count_days = calc_days
            result_calculate.append(count_days)
        
        # создаем датафрейм с результатами
        result_temp = pd.DataFrame(np.array([result_calculate]), columns = columns)
        result = result.append(result_temp, ignore_index=False)
    return result

In [4]:
calculate = min_sample_size(bcrs, uplifts, users_day, power, pvalue, ratio)
calculate.to_csv('calculate_ab_test_ratio.csv')
calculate

,conversion_name,current_rate,users_day,power,pvalue,ratio,uplift 1.0%,uplift 2.0%,uplift 3.0%,uplift 5.0%,uplift 7.0%,uplift 10.0%,uplift 15.0%,uplift 20.0%
0,CR из пользователя в просмотр карточки товара,0.3,1000,0.8,0.05,0.5,733,183,81,29,15,7,3,2
0,CR из пользователя в добавление в корзину,0.1,1000,0.8,0.05,0.5,2826,706,314,113,58,28,13,7
0,CR из пользователя в переход в корзину,0.05,1000,0.8,0.05,0.5,5965,1491,663,239,122,60,27,15
0,CR из пользователя в покопателя,0.02,1000,0.8,0.05,0.5,15384,3846,1709,615,314,154,68,38
